In [ ]:
import os,cv2
import keras
from PIL import Image
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Dropout,GlobalAveragePooling2D,Dense
from keras.preprocessing.image import ImageDataGenerator



from keras.models import Model,Sequential
from keras.layers import Dense,Flatten
from keras.layers import BatchNormalization,Activation

from scipy.misc import imread
import pickle as pickle
import matplotlib.pyplot as plt

from tqdm import tqdm

Using TensorFlow backend.


# Loading Model weights

In [ ]:
# Loading model with no weights
model_Xcep = keras.applications.xception.Xception(weights=None,include_top=False,input_shape=(299,299,3))

# Customizing network for our dataset(12 classes)
x = model_Xcep.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='sigmoid')(x)
x = Dropout(0.5)(x)
pred = Dense(12,activation='softmax')(x)
model = Model(inputs=model_Xcep.input, outputs=pred)

# Best epoch model
model.load_weights('/home/gaurav/Documents/GAURAV/Test/AML/final/model_checkpoint/patterntest.08-1.69.hdf5')

# Making Predictions

In [ ]:
pattern_dict = {0: 'Checked',1: 'Colourblock',2: 'Melange',3: 'Patterned',4: 'Printed',5: 'abstract',6: 'floral',7: 'graphic',8: 'polka dots',9: 'solid',
 10: 'striped',11: 'typography'}

In [ ]:
def crop_image(input_):
    im = Image.open(input_)
    crop_rectangle = (50, 150, 550, 600)
    cropped_im = im.crop(crop_rectangle)
    return cropped_im

In [ ]:
def image_preprocess(image):
    #cropping
    img = crop_image(image)
    #Read image
#     img = cv2.imread(image)
    img = np.array(img)
    img.resize(1,299,299,3)
    return img
    

In [ ]:
def predict(image):
    image_processed = image_preprocess(image)
    pred = model.predict(image_processed)
    y = np.argmax(pred,axis=1)
    prob = np.max(pred)
    return (pattern_dict[y[0]]),prob

In [ ]:
def show_img(path):
    img = imread(path, mode="RGB")
    plt.imshow(img)
    plt.show()
    

In [ ]:
test_result = pd.DataFrame(columns=['image_name','label'])

In [ ]:
pred_list = []
file = []
for index,i in enumerate(os.listdir('Test Directory/')):
    pred = predict('Test Directory/'+str(i))
    file.append(str(i))
    pred_list.append(pred[0])
    if index%10==0 or index%66==0:
        print('Number Processed Images: \t',index)
test_result['image_name'] = file
test_result['label'] = pred_list

In [ ]:
test_result.to_csv('test_result.csv',index=False)